In [1]:
cd D:\UCSD\WInter 2016\NN\Homework\hw2

D:\UCSD\WInter 2016\NN\Homework\hw2


In [2]:
import numpy as np
from pylab import *
import math
from scipy.special import expit
import time
import mnist
import matplotlib.pyplot as plt
from scipy.misc import logsumexp

In [3]:
#### Loading Data ######
images1,  labels1  = mnist.loadMnistofDigit('training', digits=[1,2,3,4,5,6,7,8,9,0])
images2,  labels2  = mnist.loadMnistofDigit('testing',  digits=[1,2,3,4,5,6,7,8,9,0])
L1_size = 784   # +1 for bias
#L2_size = 20    # +1 for bias
L3_size = 10
bias_size = 1    # At both Input and Hidden Layer
total_size      = 60000
training_size   = 50000
validation_size = total_size - training_size
testing_size    = 10000

#Reshaping and dividing train to validation
train_images = images1[:training_size].reshape(training_size, L1_size)
train_labels = labels1[:training_size].reshape(training_size, )

valid_images = images1[training_size:].reshape(validation_size, L1_size)
valid_labels = labels1[training_size:].reshape(validation_size, )

test_images  = images2[:testing_size].reshape(testing_size, L1_size)
test_labels  = labels2[:testing_size].reshape(testing_size, )

#Accounting for BIAS
train_images = np.insert(train_images, 0, 1, axis=1)
valid_images = np.insert(valid_images, 0, 1, axis=1)
test_images  = np.insert(test_images , 0, 1, axis=1)

In [4]:
#zscoring
train_images_mean = np.mean(train_images, axis=1).reshape(training_size, 1)
train_images_std  = np.std( train_images, axis=1).reshape(training_size, 1)
train_images      = (train_images - train_images_mean)/train_images_std

valid_images_mean = np.mean(valid_images, axis=1).reshape(validation_size, 1)
valid_images_std  = np.std( valid_images, axis=1).reshape(validation_size, 1)
valid_images      = (valid_images - valid_images_mean)/valid_images_std

test_images_mean  = np.mean(test_images, axis=1).reshape(testing_size, 1)
test_images_std   = np.std( test_images, axis=1).reshape(testing_size, 1)
test_images       = (test_images - test_images_mean)/test_images_std

In [5]:
def activation_function(A_j, fun_type):
    if fun_type == "sigmoid":
        temp  = np.matrix(expit(A_j))
    elif fun_type == "tanh":
        temp  = np.matrix(np.tanh(A_j))
    elif fun_type == "ReLU":
        temp  = np.matrix(np.maximum(np.zeros(A_j.shape), A_j))
    elif fun_type == "softmax":
        temp1 = np.matrix(logsumexp(A_j, axis=1)).transpose()
        temp2 = np.dot(temp1, np.ones((1, A_j.shape[1])))
        temp  = np.exp(A_j - temp2)
        #den  = np.sum(np.exp(A_j))
        #temp = np.matrix(np.exp(A_j))/den
    else:
        return null
    return temp

def activation_function_derivative(A_j, fun_type):
    if fun_type == "sigmoid":
        temp     =  np.matrix(expit(A_j))
        temp_bar =  np.ones(np.shape(temp)) - temp
        return np.multiply(temp, temp_bar)
    elif fun_type == "tanh":
        return np.matrix(np.multiply(1./np.cosh(A_j), 1./np.cosh(A_j)))
    elif fun_type == "ReLU":
        temp = np.matrix(np.maximum(np.zeros(A_j.shape), A_j))
        temp[np.nonzero(temp)] = 1
        return temp
    elif fun_type == "softmax":
        temp  = np.matrix(logsumexp(A_j, axis=1)).transpose()
        temp2 = np.dot(temp, np.ones((1,A_j.shape[1])))
        temp3 = np.exp(A_j-temp2)
        temp3_bar =  np.ones(np.shape(temp3)) - temp3
        return np.multiply(temp3, temp3_bar)
    else:
        return null

In [6]:
def train_NN(images, labels, ite, W_12, W_23, activation_hidden_type, activation_output_type):

    for i in range(ite):
        #----------Forward propogation-----------
        #Stochastic gradient descent: random datapoint selection
        index = np.random.randint(training_size)
        X_1   = np.matrix(images[index])
        #one-hot encoding of training data input labels
        T = np.zeros((1, L3_size))
        T[0, labels[index]] = 1.0
 
        #Hidden layer outputs using activation function
        A_2     = np.dot(X_1, W_12)
        X_2     = activation_function(A_2, activation_hidden_type)
        X_2     = np.insert(X_2, 0, 1, axis=1)  # Add BIAS term to Hidden Layer

        #Output layer outputs using activation function: softmax
        A_3     = np.dot(X_2, W_23)
        X_3     = activation_function(A_3, activation_output_type)
        
        #---------Back Propogation--------------
        #Delta at output layer
        D_3     = T - X_3
        
        #Delta at hidden layer
        D_2     = activation_function_derivative(A_2, activation_hidden_type)
        # Add BIAS term but is it needed? Since we are not using that value to update W12. Instead reduce dim(W23) ?
        D_2     = np.insert(D_2, 0, 1, axis=1)
        D_2     = np.multiply(D_2, np.dot(D_3, np.transpose(W_23)))

        #Weight matrix update at output layer with or without regularization
        if lamda == 0:
            W_23 += alpha*np.dot(np.transpose(X_2), D_3)
        else:
            W_23 = (1 - alpha*lamda)*W_23 + alpha*np.dot(np.transpose(X_2), D_3)
        
        #Weight matrix update at hidden layer with or without regularization
        if lamda == 0:
            W_12 += alpha*np.dot(np.transpose(X_1), D_2[0,1:])
        else:
            W_12 = (1 - alpha*lamda)*W_12 + alpha*np.dot(np.transpose(X_1), D_2[0,1:])

    return W_12, W_23

In [7]:
def test_NN(images, labels, data_size, W_12, W_23, activation_hidden_type, activation_output_type):

    VT   = np.zeros((data_size, L3_size))
    VT[np.arange(data_size), labels] = 1.0        

    VA_2     = np.dot(images, W_12)
    VX_2     = activation_function(VA_2, activation_hidden_type)
    VX_2     = np.insert(VX_2, 0, 1, axis=1) # Add BIAS term
    VA_3     = np.dot(VX_2, W_23)
    VX_3     = activation_function(VA_3, activation_output_type)

    # Error (cost) computation with or without regularization
    E        = sum(np.multiply(VT, np.log(VX_3 + ((10**-40)*np.ones(VX_3.shape)))))
    if lamda != 0:
        E   += lamda*(np.sum(np.multiply(W_12, W_12)) + np.sum(np.multiply(W_23, W_23)))/2    

    VY       = np.argmax(VX_3, axis=1)
    Err      = np.count_nonzero(VY.reshape(np.shape(labels)) - labels)
    return E, Err

In [8]:
#### Initialization Block ####

#alphas = [0.001, 0.0025, 0.005, 0.0075, 0.01, 0.015]
#L2_sizes = [50, 100, 150, 200, 250, 300, 350]
#activation_list = ["sigmoid", "tanh", "ReLU", "softmax"]
#alphas   = [0.006]   # Learning Rate

alphas = [0.002, 0.004, 0.006, 0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.020]
activation_list = ["sigmoid", "tanh", "softmax"]
L2_size  = 150   # Number of Hidden Nodes
lamda    = 0    # For regularization
gamma    = 0    # For momentum
activation_function_Hidden = activation_list[0]
activation_function_Output = "softmax"

if activation_function_Hidden == "ReLU":
    training_block_size = 10
else:
    training_block_size = int(training_size/2)

InitWeight_12     = 0.01*np.random.rand(L1_size + bias_size, L2_size)
InitWeight_23     = 0.01*np.random.rand(L2_size + bias_size, L3_size)
Convergence_check = 5
Convergence_count = 3
testErrorAlways   = 0

In [ ]:
TrE       = []
VE        = []
TE        = []
iterCount = []
TrEFinal  = []
VEFinal   = []
TEFinal   = []
W12Final  = []
W23Final  = []
iterCountFinal = []

for activation_function_Hidden in activation_list:
    for alpha in alphas:
        print("### Alpha = ", alpha, " Activation Function Hidden = ", activation_function_Hidden, " and # Hidden Nodes = ", L2_size)
        iter_count = 0
        conv_count = 0
        VE = []
        TE = []
        iterCount = []

        weight_12 = np.copy(InitWeight_12)
        weight_23 = np.copy(InitWeight_23)

        ETr, ErrTr = test_NN(train_images, train_labels, training_size,   weight_12, weight_23, activation_function_Hidden, activation_function_Output)
        EVl, ErrVl = test_NN(valid_images, valid_labels, validation_size, weight_12, weight_23, activation_function_Hidden, activation_function_Output)
        print("Iteration = ", iter_count, " Training   Error = ", ETr, " :: Error Rate = ", ErrTr*100/training_size)
        print("Iteration = ", iter_count, " Validation Error = ", EVl, " :: Error Rate = ", ErrVl*100/validation_size)

        if testErrorAlways == 1:
            ETest, ErrTest = test_NN(test_images, test_labels, testing_size, weight_12, weight_23, activation_function_Hidden, activation_function_Output)
            print("Error for Test Images = ", ETest, " :: Error Rate = ", ErrTest*100/testing_size)
            TE.append(100 - ErrTest*100/testing_size)

        TrE.append(100 - ErrTr*100/training_size)
        VE.append(100 - ErrVl*100/validation_size)
        iterCount.append(iter_count*training_block_size)

        delta_E = abs(EVl)
        while delta_E > Convergence_check or conv_count < Convergence_count:
            weight_12, weight_23 = train_NN(train_images, train_labels, training_block_size, weight_12, weight_23, activation_function_Hidden, activation_function_Output)

            iter_count += 1
            E_initial  = EVl
            ETr, ErrTr = test_NN(train_images, train_labels, training_size,   weight_12, weight_23, activation_function_Hidden, activation_function_Output)
            EVl, ErrVl = test_NN(valid_images, valid_labels, validation_size, weight_12, weight_23, activation_function_Hidden, activation_function_Output)
            print("Iteration = ", iter_count, " Training   Error = ", ETr, " :: Error Rate = ", ErrTr*100/training_size)
            print("Iteration = ", iter_count, " Validation Error = ", EVl, " :: Error Rate = ", ErrVl*100/validation_size)
            if testErrorAlways == 1:
                ETest, ErrTest = test_NN(test_images, test_labels, testing_size, weight_12, weight_23, activation_function_Hidden, activation_function_Output)
                print("Error for Test Images = ", ETest, " :: Error Rate", ErrTest*100/testing_size)
                TE.append(100 - ErrTest*100/testing_size)

            TrE.append(100 - ErrTr*100/training_size)
            VE.append(100 - ErrVl*100/validation_size)
            iterCount.append(iter_count*training_block_size)

            delta_E = abs(E_initial - EVl)
            if delta_E <= Convergence_check:
                conv_count += 1
            else:
                conv_count = 0


        print("Training   Error = {0} :: Error Rate = {1} :: Number of iterations = {2}" .format(ETr, ErrTr*100/training_size,   iter_count*training_block_size))
        print("Validation Error = {0} :: Error Rate = {1} :: Number of iterations = {2}" .format(EVl, ErrVl*100/validation_size, iter_count*training_block_size))
        if testErrorAlways == 0:
            ETest, ErrTest = test_NN(test_images, test_labels, testing_size, weight_12, weight_23, activation_function_Hidden, activation_function_Output)
            print("Test Error       = ", ETest, " :: Error Rate = ", ErrTest*100/testing_size)
            TE.append(100 - ErrTest*100/testing_size)
        TrEFinal.append(TrE)
        VEFinal.append(VE)
        TEFinal.append(TE)
        W12Final.append(weight_12)
        W23Final.append(weight_23)        
        iterCountFinal.append(iterCount)        


### Alpha =  0.002  Activation Function Hidden =  sigmoid  and # Hidden Nodes =  150
Iteration =  0  Training   Error =  -115100.841501

In [48]:
# File Write
np.set_printoptions(threshold=200000)
logs = open("Solutions_1.txt", 'w')
logs.write("TrEFinal\n" + str(TrEFinal))
logs.write("VEFinal\n" + str(VEFinal))
logs.write("TEFinal\n" + str(TEFinal))
logs.write("W12Final\n" + str(W12Final[0]))
logs.write("W23Final\n" + str(W23Final[0]))
logs.write("iterCountFinal\n" + str(iterCountFinal))
logs.close()

In [58]:
n = argmax(VEFinal[0])
print(len(VEFinal[0]), VEFinal[0][-1], TrEFinal[0][-1])
print(n, VEFinal[0][n], TrEFinal[0][n])

297 97.99 99.996
148 98.06 99.902


In [55]:
# Plotting Block
plt.title('Alpha =  0.001, Activation Function Hidden = sigmoid, and # Hidden Nodes = 150')
plt.plot(iterCountFinal[0][1:], TrEFinal[0][1:], color="blue", label="Training")
plt.plot(iterCountFinal[0][1:], VEFinal[0][1:],  color="red" , label="Validation")
plt.ylabel('Training (b) / Validation (r) Accuracy Rates')
plt.xlabel('Iteration Count')
plt.grid(True)
plt.legend(loc='lower right')
plt.show()